In [51]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
import pandas as pd
from sklearn.model_selection import KFold
#OLS
from sklearn.linear_model import LinearRegression
import numpy as np
from IPython.display import HTML, display
import matplotlib.pyplot as plt
%matplotlib inline


In [52]:
#Load data
data = pd.read_csv('rent_clean.csv')
test = data[['price_per_sqm', 'apartment', 'number_of_whole_rooms', 'floor', 'number_of_half_rooms', 'furnish_furnished', 'furnish_partly_furnished', 'view_garden', 'view_panoramic', 'view_street', 'balcony', 'lift', 'air_conditioner', 'utility_double_comfort', 'heating_gas', 'heating_wall_heating', 'heating_combination', 'heating_ceiling', 'heating_central_heating', 'heating_electric', 'heating_floor', 'heating_circulating', 'heating_district', 'available_immediately', 'condition_of_property_renovated', 'parking']]
test = test.dropna()


/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (39,50,111) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [53]:
X = test[['number_of_whole_rooms','number_of_half_rooms','furnish_furnished','view_panoramic','lift','air_conditioner', 'heating_district']]
y = test[['price_per_sqm']]


In [54]:
model = Pipeline([('poly', PolynomialFeatures(degree=2)), ('linear', LinearRegression(fit_intercept=False))])
model = model.fit(X,y)
model.named_steps['linear'].coef_
    

array([[ 1.22083327e+01, -2.43011818e+00, -4.39619756e-03,
         3.97213905e-01,  6.92404698e-02,  9.20331293e-02,
         8.07991261e-02,  2.29804649e-01,  4.13485935e-01,
        -1.35156850e-01,  1.09394759e-01,  4.07009955e-02,
         4.42668330e-01,  3.59765325e-01, -1.13065003e+00,
         6.58133184e-02, -4.02340452e-01, -8.35120691e-01,
        -3.69485618e-01,  1.18081680e-01, -3.94044498e-01,
         3.97213905e-01, -6.98451412e-01,  5.39049690e-01,
         4.28246026e-01, -1.10405931e+00,  6.92404698e-02,
         3.69415330e-01,  1.95750012e+00,  4.05016684e-01,
         9.20331293e-02,  3.89725074e-01,  2.28938621e-01,
         8.07991261e-02, -5.08796763e-01,  2.29804649e-01]])

In [56]:
#10 fold
scores = []
cv = KFold(n_splits = 10, random_state = 42, shuffle = False)
for train_index, test_index in cv.split(X):
    #print('Train Index:', train_index, '\n')
    #print('Test Index:', test_index)

    X_train, X_test, y_train, y_test = X.iloc[train_index.tolist()], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    my_model = Pipeline([('poly', PolynomialFeatures(degree=2)), ('linear', LinearRegression(fit_intercept=False))])
    my_model = model.fit(X_train,y_train)
    scores.append(my_model.score(X_test, y_test))

r_square = np.mean(scores)
adj_r = 1-(1-r_square)*(len(y)-1)/(len(y)-X.shape[1]-1)

print('R Square:', r_square, '\n')
print('Adjusted R Square:', adj_r, '\n')


R Square: 0.1823681885252101 

Adjusted R Square: 0.17780041304211058 



In [63]:
housing_model = ols("price_per_sqm ~ number_of_whole_rooms + I(number_of_whole_rooms**2) + number_of_half_rooms + I(number_of_half_rooms**2)+ furnish_furnished + I(furnish_furnished**2)+ view_panoramic + I(view_panoramic**2) + lift + I(lift**2) + air_conditioner + I(air_conditioner**2) + heating_district + I(heating_district**2)", data=data).fit()

# summarize our model
housing_model_summary = housing_model.summary()

# convert our table to HTML and add colors to headers for explanatory purposes
HTML(
(housing_model_summary
    .as_html()
    .replace('<th>  Adj. R-squared:    </th>', '<th style="background-color:#aec7e8;"> Adj. R-squared: </th>')
    .replace('<th>coef</th>', '<th style="background-color:#ffbb78;">coef</th>')
    .replace('<th>std err</th>', '<th style="background-color:#c7e9c0;">std err</th>')
    .replace('<th>P>|t|</th>', '<th style="background-color:#bcbddc;">P>|t|</th>')
    .replace('<th>[0.025</th>    <th>0.975]</th>', '<th style="background-color:#ff9896;">[0.025</th>    <th style="background-color:#ff9896;">0.975]</th>'))
)